In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
dataset_path = '/content/gdrive/My Drive/project/dataset/'

# Actual Code Starting Point

## Library Imports

In [ ]:
import os
import pandas as pd
import numpy as np

## Creating a single Data Frame using all Dataset files

In [ ]:
all_files = [dataset_path+'original/'+each_file for each_file in os.listdir(dataset_path+'original/') if each_file != '.DS_Store']
all_dfs = [pd.read_csv(each_file,encoding='cp1252') for each_file in all_files]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (0,1,3,6,84) have mixed types.Specify dtype option on import or set low_memory=False.
  


In [ ]:
total_columns = all_dfs[0].columns
all_same_column = np.array([])
for (index,df) in enumerate(all_dfs):
  all_same_column = df.columns == total_columns
  if False in all_same_column:
    print(f"This {all_files[index]} doesn't have the same columns")

In [ ]:
if np.all(all_same_column):
  print("All files have same columns")
  merge_df  = pd.concat([each for each in all_dfs]).drop_duplicates(keep=False)
  merge_df.reset_index(drop=True, inplace = True)
  print("Total Data Shape :" +str(merge_df.shape))
else:
  print("All files have not same columns")

All files have same columns
Total Data Shape :(2830434, 85)


In [ ]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830434 entries, 0 to 2830433
Data columns (total 85 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Flow ID                       object 
 1    Source IP                    object 
 2    Source Port                  float64
 3    Destination IP               object 
 4    Destination Port             float64
 5    Protocol                     float64
 6    Timestamp                    object 
 7    Flow Duration                float64
 8    Total Fwd Packets            float64
 9    Total Backward Packets       float64
 10  Total Length of Fwd Packets   float64
 11   Total Length of Bwd Packets  float64
 12   Fwd Packet Length Max        float64
 13   Fwd Packet Length Min        float64
 14   Fwd Packet Length Mean       float64
 15   Fwd Packet Length Std        float64
 16  Bwd Packet Length Max         float64
 17   Bwd Packet Length Min        float64
 18   Bwd Packet Length Mea

In [ ]:
merge_df.head()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.5-8.254.250.126-49188-80-6,8.254.250.126,80.0,192.168.10.5,49188.0,6.0,03/07/2017 08:55:58,4.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.14-8.253.185.121-49486-80-6,8.253.185.121,80.0,192.168.10.14,49486.0,6.0,03/07/2017 08:56:22,3.0,2.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.3-192.168.10.9-88-1031-6,192.168.10.9,1031.0,192.168.10.3,88.0,6.0,03/07/2017 08:56:38,609.0,7.0,4.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.3-192.168.10.9-88-1032-6,192.168.10.9,1032.0,192.168.10.3,88.0,6.0,03/07/2017 08:56:38,879.0,9.0,4.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.3-192.168.10.9-88-1033-6,192.168.10.9,1033.0,192.168.10.3,88.0,6.0,03/07/2017 08:56:38,1160.0,9.0,6.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


## Initial Cleaning

## Deleting columns with ZERO std.

In [ ]:
# Deleting columns with ZERO std.
stats = merge_df.describe()
std = stats.loc['std']
features_no_variance = std[std == 0.0].index
features_no_variance

Index([' Bwd PSH Flags', ' Bwd URG Flags', 'Fwd Avg Bytes/Bulk',
       ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk',
       ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate'],
      dtype='object')

In [ ]:
print("Datashape before removal")
print(merge_df.shape)
merge_df = merge_df.drop(columns=features_no_variance)
print("Datashape after removal")
print(merge_df.shape)

Datashape before removal
(2830434, 85)
Datashape after removal
(2830434, 77)


## Removing non-numeric columns

In [ ]:
# Removing non-numeric columns
discriptive_stat = merge_df.describe()
str_cols = list(set(merge_df.columns)-set(discriptive_stat.columns))
str_cols.remove(' Label')
print(str_cols)

['Flow ID', ' Destination IP', ' Source IP', ' Timestamp']


In [ ]:
merge_df = merge_df.drop(columns=str_cols)
print("Datashape after removal")
print(merge_df.shape)

Datashape after removal
(2830434, 73)


## Removing unnecessary columns

In [ ]:
# Removing unnecessary columns
del_columns = [' Source Port', ' Protocol']
merge_df = merge_df.drop(columns=del_columns)
print("Datashape after removal")
print(merge_df.shape)

Datashape after removal
(2830434, 71)


In [ ]:
merge_df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188.0,4.0,2.0,0.0,12.0,0.0,6.0,6.0,6.000000,0.000000,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,49486.0,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.000000,0.000000,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,88.0,609.0,7.0,4.0,484.0,414.0,233.0,0.0,69.142857,111.967895,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,88.0,879.0,9.0,4.0,656.0,3064.0,313.0,0.0,72.888889,136.153814,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,88.0,1160.0,9.0,6.0,3134.0,3048.0,1552.0,0.0,348.222222,682.482560,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


## Removing Infinit & Null Values

In [ ]:
print(f'Total row before removing null and infinite value are ', list(merge_df.shape)[0])

Total row before removing null and infinite value are  2830434


In [ ]:
labl = merge_df[' Label']
dataset = merge_df.loc[:, merge_df.columns != ' Label'].astype('float64')

In [ ]:
# Checking what column/s contain non-finite values.
nonfinite = [col for col in dataset if not np.all(np.isfinite(dataset[col]))]
print(nonfinite)

['Flow Bytes/s', ' Flow Packets/s']


In [ ]:
# Checking how many non-finite values each column contains.
finite = np.isfinite(merge_df['Flow Bytes/s']).sum()
print("FlowBytes/s: "+str(merge_df.shape[0] - finite))
# Checking how many non-finite values each column contains.
finite = np.isfinite(merge_df[' Flow Packets/s']).sum()
print("Flow Packets/s  "+str(merge_df.shape[0] - finite))

FlowBytes/s: 2859
Flow Packets/s  2859


In [ ]:
# Same as before, since there is a small number of non-finite values we can safely remove them from the dataset Replacing infinite values with NaN values.
merge_df = merge_df.replace([np.inf, -np.inf], np.nan)

In [ ]:
merge_df.isnull().values.any()

True

In [ ]:
# Total number of rows containing NAN values
print(f"Total number of rows containing NAN values: {merge_df.shape[0] - merge_df.dropna().shape[0]}")

Total number of rows containing NAN values: 2859


In [ ]:
# Considering that only 5718 rows contain NULL vlaues in the entire dataset, which makes about 0.10%, we
# can safely remove all NULL rows without spoiling the data.
perct = 2859/merge_df.shape[0]*100
print(f'{perct} of dataset will be removed')

0.1010092445186851 of dataset will be removed


In [ ]:
# Removing rows that contain NULL values and checking if number of removed rows is equal to the number of null values.
before = merge_df.shape
merge_df.dropna(inplace=True)
after = merge_df.shape
print("No of removed rows: "+str(before[0] - after[0]))

No of removed rows: 2859


## Removing Duplicates

In [ ]:
before = merge_df.shape
# drop duplicate rows
merge_df =  merge_df.drop_duplicates(keep="first")
merge_df.reset_index(drop=True,inplace=True)
after = merge_df.shape
print("No of removed duplicate rows: "+str(before[0] - after[0]))

No of removed duplicate rows: 306664


# Removing Features with same single values

In [ ]:
# drop one variable features 
one_variable_list = []
for i in merge_df.columns:
    if merge_df[i].value_counts().nunique() < 2:
        one_variable_list.append(i)
        
print(one_variable_list)

[]


## Categorical Features Encoding

# Normalizing Column Names and Labels

> Indented block



In [ ]:
# Removing whitespaces in column names.
col_names = [col.replace(' ', '').replace('-','').replace('/','').replace('\\','') for col in merge_df.columns]
merge_df.columns = col_names
merge_df.head(2)

,DestinationPort,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMax,FwdPacketLengthMin,FwdPacketLengthMean,FwdPacketLengthStd,...,min_seg_size_forward,ActiveMean,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin,Label
0,49188.0,4.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,49486.0,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [ ]:
# Dealing With Labels Classes removing space and hyphens
def normalizeClass(label):
  if label == 'BENIGN':
    return 'normal'
  else:
    return label.replace(' ','').replace('–','').replace('-','')
merge_df['Label'] = merge_df['Label'].apply(normalizeClass)
merge_df.groupby('Label').agg({'Label':'count'}).rename_axis(None).sort_values('Label', ascending=False)

,Label
normal,2095170
DoSHulk,172846
DDoS,128014
PortScan,90694
DoSGoldenEye,10286
FTPPatator,5931
DoSslowloris,5385
DoSSlowhttptest,5228
SSHPatator,3219
Bot,1948


In [ ]:
merge_df = pd.read_csv(dataset_path+'processed/cleanDatasetWithoutEncoding.csv')

## Memory Reduction by type casting

In [ ]:
old_memory_usage = merge_df.memory_usage().sum()
#change the variable types for low memory usage
#int64 to int32,,, float64 to float32
integer = []
f = []
for i in merge_df.columns[:-1]:
    if merge_df[i].dtype == "int64": integer.append(i)
    else : f.append(i)

merge_df[integer] = merge_df[integer].astype("int32")
merge_df[f] = merge_df[f].astype("float32")
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2520911 entries, 0 to 2520910
Data columns (total 71 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   DestinationPort          float32
 1   FlowDuration             float32
 2   TotalFwdPackets          float32
 3   TotalBackwardPackets     float32
 4   TotalLengthofFwdPackets  float32
 5   TotalLengthofBwdPackets  float32
 6   FwdPacketLengthMax       float32
 7   FwdPacketLengthMin       float32
 8   FwdPacketLengthMean      float32
 9   FwdPacketLengthStd       float32
 10  BwdPacketLengthMax       float32
 11  BwdPacketLengthMin       float32
 12  BwdPacketLengthMean      float32
 13  BwdPacketLengthStd       float32
 14  FlowBytess               float32
 15  FlowPacketss             float32
 16  FlowIATMean              float32
 17  FlowIATStd               float32
 18  FlowIATMax               float32
 19  FlowIATMin               float32
 20  FwdIATTotal              float32
 21  FwdIATMe

In [ ]:
new_memory_usage = merge_df.memory_usage().sum()
old_vs_new = (old_memory_usage - new_memory_usage) / old_memory_usage * 100
print(f"%{old_vs_new} lower memory usage")

%49.295770241184364 lower memory usage


## Saving Dataset as CSV for later Use

In [ ]:
# Now we are saving dataset such that labels are encoded
merge_df.to_csv(dataset_path+'processed/cleanDatasetWithoutEncoding.csv', index=False)

# Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Seperating labels from features
labels = merge_df['Label']
print(labels)

0          normal
1          normal
2          normal
3          normal
4          normal
            ...  
2520906    normal
2520907    normal
2520908    normal
2520909    normal
2520910    normal
Name: Label, Length: 2520911, dtype: object


In [ ]:
# Labels have been replaced with integers.
LE = LabelEncoder()
LE.fit(labels)
encodedLabels = LE.transform(labels)
print(encodedLabels)

[14 14 14 ... 14 14 14]


In [ ]:
# Replacing string label with equivalent number labels
merge_df['Label'] = LE.fit_transform(merge_df['Label'])
merge_df.head()

,DestinationPort,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMax,FwdPacketLengthMin,FwdPacketLengthMean,FwdPacketLengthStd,...,min_seg_size_forward,ActiveMean,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin,Label
0,49188.0,4.0,2.0,0.0,12.0,0.0,6.0,6.0,6.000000,0.000000,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14
1,49486.0,3.0,2.0,0.0,12.0,0.0,6.0,6.0,6.000000,0.000000,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14
2,88.0,609.0,7.0,4.0,484.0,414.0,233.0,0.0,69.142860,111.967896,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14
3,88.0,879.0,9.0,4.0,656.0,3064.0,313.0,0.0,72.888885,136.153809,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14
4,88.0,1160.0,9.0,6.0,3134.0,3048.0,1552.0,0.0,348.222229,682.482544,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14


In [ ]:
# Now we are saving dataset such that labels are encoded
merge_df.to_csv(dataset_path+'processed/cleanDatasetWithEncoding.csv', index=False)